# Protypical Networks

Keras implementation

In [1]:
%pylab inline

import os
import logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.INFO, format='%(message)s')
# logging.basicConfig(level=logging.INFO)
Logger = logging.getLogger('protonet')
Logger.setLevel(logging.INFO)

# import skimage, skimage.io, skimage.transform

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow as tf

from tensorflow import keras, nn

2021-08-20 00:11:26.997175: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
import fsl

## Load data

In [4]:
datadir = os.path.expanduser('~/Database/Machine_Learning')
workdir = os.getcwd()

outdir = f'{workdir}/outputs'

try:
    os.makedirs(outdir)
except Exception as msg:
    print(msg)
    pass

[Errno 17] File exists: '/home/han/Projects/Few Shot Learning/Notebooks/Prototype_Networks/outputs'


### Data loader

In [5]:
# ## On the original dataset
# indir = f'{datadir}/Omniglot/omniglot.original/python/'

# foo1 = nd.from_folder(f'{indir}/images_background')
# foo2 = nd.from_folder(f'{indir}/images_evaluation')

# data_dict = {'images_background': foo1, 'images_evaluation': foo2}
# # print(len(data_dict.keys()))

# ms = fsl.utils.data.Meta_Sampler(data_dict, 0.75, lvl_split=1, lvl_task=2, n_way=20, k_shot=5)

# print(len(ms.trainval_dict.keys()))

# print(len(ms.test_dict.keys()))

In [6]:
## On the fused dataset
indir = f'{datadir}/Omniglot/omniglot.fused/data/'
train_dict, test_dict = fsl.utils.sampler.split_dict_from_folder(indir, 0.75, 0)

# data_dict = fsl.utils.nested_dict.from_folder(indir)
# train_dict, test_dict = fsl.utils.nested_dict.random_split_level(data_dict, 0.75, 0)

# train_sampler = fsl.utils.sampler.Meta_Sampler(train_dict, indir, lvl_task=1, n_way=10, k_shot=5, q_shot=None)
# test_sampler = fsl.utils.sampler.Meta_Sampler(test_dict, indir, lvl_task=1, n_way=10, k_shot=5, q_shot=None)
# test_sampler = train_sampler

In [7]:
kwargs_imread = dict(invert=True, dim=(20,20))

func = lambda x: fsl.utils.data.imread(x, **kwargs_imread)
train_sampler = fsl.utils.sampler.SequentialSampler(train_dict, indir, lvl_task=1, \
                                               n_way=10, k_shot=5, q_shot=None, local=False, func=func, split=True)
test_sampler = fsl.utils.sampler.SequentialSampler(test_dict, indir, lvl_task=1, \
                                               n_way=10, k_shot=5, q_shot=None, local=False, func=func, split=True)
# train_sampler = fsl.utils.sampler.Meta_Sampler(train_dict, indir, lvl_task=1)


## Model construction

In [14]:
# model = fsl.models.tf.ProtoNet((*kwargs_imread['dim'],1))
model = fsl.models.tf.ProtoNet((*kwargs_imread['dim'],1), train_sampler.n_way, train_sampler.k_shot)
# model = fsl.models.tf.get_model(kwargs_imread['dim'])

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseTopKCategoricalAccuracy(k=1)],
              optimizer=keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

In [15]:
res = model.fit(train_sampler, validation_data=test_sampler, epochs=100, use_multiprocessing=True)

Epoch 1/1000
122/122 [==============================] - 39s 307ms/step - loss: 0.8629 - sparse_top_k_categorical_accuracy: 0.7571 - val_loss: 2.1371 - val_sparse_top_k_categorical_accuracy: 0.6443
Epoch 2/1000
122/122 [==============================] - 40s 326ms/step - loss: 0.3629 - sparse_top_k_categorical_accuracy: 0.8824 - val_loss: 0.7465 - val_sparse_top_k_categorical_accuracy: 0.8310
Epoch 3/1000
122/122 [==============================] - 38s 309ms/step - loss: 0.2578 - sparse_top_k_categorical_accuracy: 0.9120 - val_loss: 0.3890 - val_sparse_top_k_categorical_accuracy: 0.8837
Epoch 4/1000
122/122 [==============================] - 38s 309ms/step - loss: 0.2072 - sparse_top_k_categorical_accuracy: 0.9332 - val_loss: 0.4151 - val_sparse_top_k_categorical_accuracy: 0.8885
Epoch 5/1000
122/122 [==============================] - 34s 273ms/step - loss: 0.1775 - sparse_top_k_categorical_accuracy: 0.9407 - val_loss: 0.3596 - val_sparse_top_k_categorical_accuracy: 0.9010
Epoch 6/1000
12

122/122 [==============================] - 40s 323ms/step - loss: 0.0322 - sparse_top_k_categorical_accuracy: 0.9889 - val_loss: 0.1694 - val_sparse_top_k_categorical_accuracy: 0.9570
Epoch 43/1000
122/122 [==============================] - 37s 303ms/step - loss: 0.0343 - sparse_top_k_categorical_accuracy: 0.9877 - val_loss: 0.1347 - val_sparse_top_k_categorical_accuracy: 0.9612
Epoch 44/1000
122/122 [==============================] - 30s 247ms/step - loss: 0.0326 - sparse_top_k_categorical_accuracy: 0.9883 - val_loss: 0.1628 - val_sparse_top_k_categorical_accuracy: 0.9610
Epoch 45/1000
122/122 [==============================] - 23s 187ms/step - loss: 0.0325 - sparse_top_k_categorical_accuracy: 0.9884 - val_loss: 0.1648 - val_sparse_top_k_categorical_accuracy: 0.9585
Epoch 46/1000
122/122 [==============================] - 23s 186ms/step - loss: 0.0334 - sparse_top_k_categorical_accuracy: 0.9887 - val_loss: 0.1291 - val_sparse_top_k_categorical_accuracy: 0.9655
Epoch 47/1000
122/122 [=

122/122 [==============================] - 24s 198ms/step - loss: 0.0246 - sparse_top_k_categorical_accuracy: 0.9913 - val_loss: 0.1125 - val_sparse_top_k_categorical_accuracy: 0.9710
Epoch 84/1000
122/122 [==============================] - 24s 196ms/step - loss: 0.0166 - sparse_top_k_categorical_accuracy: 0.9943 - val_loss: 0.1295 - val_sparse_top_k_categorical_accuracy: 0.9692
Epoch 85/1000
122/122 [==============================] - 25s 202ms/step - loss: 0.0200 - sparse_top_k_categorical_accuracy: 0.9923 - val_loss: 0.1098 - val_sparse_top_k_categorical_accuracy: 0.9737
Epoch 86/1000
122/122 [==============================] - 24s 198ms/step - loss: 0.0191 - sparse_top_k_categorical_accuracy: 0.9931 - val_loss: 0.1181 - val_sparse_top_k_categorical_accuracy: 0.9672
Epoch 87/1000
122/122 [==============================] - 24s 193ms/step - loss: 0.0171 - sparse_top_k_categorical_accuracy: 0.9942 - val_loss: 0.1141 - val_sparse_top_k_categorical_accuracy: 0.9703
Epoch 88/1000
122/122 [=

122/122 [==============================] - 23s 190ms/step - loss: 0.0125 - sparse_top_k_categorical_accuracy: 0.9954 - val_loss: 0.1079 - val_sparse_top_k_categorical_accuracy: 0.9740
Epoch 125/1000
122/122 [==============================] - 24s 199ms/step - loss: 0.0128 - sparse_top_k_categorical_accuracy: 0.9950 - val_loss: 0.1310 - val_sparse_top_k_categorical_accuracy: 0.9748
Epoch 126/1000
122/122 [==============================] - 29s 241ms/step - loss: 0.0129 - sparse_top_k_categorical_accuracy: 0.9957 - val_loss: 0.1077 - val_sparse_top_k_categorical_accuracy: 0.9748
Epoch 127/1000
122/122 [==============================] - 28s 230ms/step - loss: 0.0115 - sparse_top_k_categorical_accuracy: 0.9963 - val_loss: 0.1158 - val_sparse_top_k_categorical_accuracy: 0.9712
Epoch 128/1000
122/122 [==============================] - 23s 184ms/step - loss: 0.0184 - sparse_top_k_categorical_accuracy: 0.9931 - val_loss: 0.1179 - val_sparse_top_k_categorical_accuracy: 0.9753
Epoch 129/1000
122/1

122/122 [==============================] - 23s 190ms/step - loss: 0.0119 - sparse_top_k_categorical_accuracy: 0.9959 - val_loss: 0.1028 - val_sparse_top_k_categorical_accuracy: 0.9790
Epoch 166/1000
122/122 [==============================] - 23s 189ms/step - loss: 0.0090 - sparse_top_k_categorical_accuracy: 0.9963 - val_loss: 0.1456 - val_sparse_top_k_categorical_accuracy: 0.9687
Epoch 167/1000
122/122 [==============================] - 24s 194ms/step - loss: 0.0128 - sparse_top_k_categorical_accuracy: 0.9951 - val_loss: 0.0988 - val_sparse_top_k_categorical_accuracy: 0.9812
Epoch 168/1000
122/122 [==============================] - 29s 240ms/step - loss: 0.0099 - sparse_top_k_categorical_accuracy: 0.9962 - val_loss: 0.1234 - val_sparse_top_k_categorical_accuracy: 0.9742
Epoch 169/1000
122/122 [==============================] - 37s 298ms/step - loss: 0.0092 - sparse_top_k_categorical_accuracy: 0.9962 - val_loss: 0.1443 - val_sparse_top_k_categorical_accuracy: 0.9688
Epoch 170/1000
122/1

KeyboardInterrupt: 

In [16]:
model.save(f'{outdir}/model_tf.h5')

In [ ]:
res = model.evaluate(test_sampler, steps=1000)

In [1]:
import pydot

keras.utils.plot_model(model, f"{outdir}/model.png", show_shapes=True)

NameError: name 'keras' is not defined

## EOF

In [ ]:
S = tf.convert_to_tensor(np.random.randn(10,5,100,100,1))
Q = tf.convert_to_tensor(np.random.randn(10,15,100,100,1))

Y = model([Q, S])

Yp = tf.nn.softmax(Y, axis=-1)

In [ ]:
n_way = 10
q_shot = 15

scce = tf.keras.losses.SparseCategoricalCrossentropy()

Yt = np.tile(np.arange(n_way), (q_shot,1)).T.flatten()

scce(Yt, Yp)

In [ ]:
n_way = 10
q_shot = 15

scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Yt = np.tile(np.arange(n_way), (q_shot,1)).T.flatten()

scce(Yt, Y)

In [ ]:
Yt = np.tile(np.arange(n_way)[:,None], (1,q_shot))  # target labels

Yt = tf.reshape(tf.one_hot(Yt, depth=n_way, axis=-1), (-1,n_way))

# cce = np.mean(keras.losses.categorical_crossentropy(Yt, Yp, from_logits=False))

cce = tf.keras.losses.CategoricalCrossentropy()
cce(Yt, Yp)

In [ ]:
acc = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1)
acc.update_state(Yt, Yp)
acc.result().numpy()